In [2]:
!pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 6.4 MB/s eta 0:00:00


In [14]:
import gymnasium as gym
import numpy as np
import time
from IPython import display
import time

# Một số hàm dùng để chơi game

In [4]:
def play(env, policy, render=False):
    state, _ = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info, _ = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            print(env.render())
            time.sleep(0.5)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [5]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [6]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value

        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

# Cài đặt thuật toán Value Iteration

In [7]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []

            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])

                q_values.append(q_value)

            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]

        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [8]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int32)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])

            q_values.append(q_value)

        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action

    return policy

# Cài đặt thuật toán Policy Iteration

In [9]:
def sum_sr(env, V, s, a, gamma):
    """Calc state-action value for state 's' and action 'a'"""
    tmp = 0  # state value for state s
    for p, s_, r, _ in env.P[s][a]:
        tmp += p * (r + gamma * V[s_])
    return tmp

In [10]:
def policy_iteration(env, max_iter = 500, gamma=0.9, theta=1e-8):
    # initialization
    v_values = np.zeros(env.observation_space.n)
    pi = np.zeros(env.observation_space.n, dtype=int)
    # policy Evaluation
    iter = 0
    while iter<max_iter:
      iter+=1
      while True:
        delta = 0
        for s in range(env.observation_space.n):
          v = v_values[s]
          v_values[s] = sum_sr(env,V=v_values, s=s, a=pi[s], gamma=gamma)
          delta = max(delta, abs(v-v_values[s]))
        if delta < theta: break
    # policy Improvement
      policy_stable = True
      for s in range(env.observation_space.n):
        pre_action = pi[s]
        pi[s] = np.argmax([sum_sr(env, V=v_values, s=s, a=a, gamma=gamma) for a in range(env.action_space.n)])
        if pre_action != pi[s]: policy_stable = False
      if policy_stable:
        print(f'Converged at {iter}-th iteration.')
        break
    return v_values, pi

# Compare Model

## FrozenLake-v1

In [13]:
env = gym.make('FrozenLake-v1', render_mode="ansi")
print(env.P[0][3])
print(env.observation_space.n, env.action_space.n)

[(0.3333333333333333, 1, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 0, 0.0, False)]
16 4


In [15]:
# Value Iteration
print('FROZEN-LAKE-V1 USING VALUE ITERATION')
s = time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)
play_multiple_times(env, optimal_policy, 1000)
e = time.time()
print('Executable time:',e-s)

FROZEN-LAKE-V1 USING VALUE ITERATION
Converged at 79-th iteration.
Number of successes: 769/1000
Average number of steps: 44.57867360208063
Executable time: 0.9382474422454834


In [16]:
# Policy Iteration
print('FROZEN-LAKE-V1 USING POLICY ITERATION')
s = time.time()
optimal_p_values = policy_iteration(env)[1]
play_multiple_times(env, optimal_p_values, 1000)
e = time.time()
print('Executable time:',e-s)

FROZEN-LAKE-V1 USING POLICY ITERATION
Converged at 6-th iteration.
Number of successes: 765/1000
Average number of steps: 42.94640522875817
Executable time: 0.6238429546356201


## FrozenLake8x8-v1

In [18]:
env = gym.make('FrozenLake8x8-v1')
print(env.P[0][3])
print(env.observation_space.n, env.action_space.n)

[(0.3333333333333333, 1, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 0, 0.0, False)]
64 4


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


In [19]:
# Value Iteration
print('FROZEN-LAKE-8X8-V1 USING VALUE ITERATION')
s = time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)
play_multiple_times(env, optimal_policy, 1000)
e = time.time()
print('Executable time:',e-s)

FROZEN-LAKE-8X8-V1 USING VALUE ITERATION
Converged at 117-th iteration.
Number of successes: 790/1000
Average number of steps: 75.96455696202531
Executable time: 2.029763698577881


In [20]:
# Policy Iteration
print('FROZEN-LAKE-8X8-V1 USING POLICY ITERATION')
s = time.time()
optimal_p_values = policy_iteration(env)[1]
play_multiple_times(env, optimal_p_values, 1000)
e = time.time()
print('Executable time:',e-s)

FROZEN-LAKE-V1 USING POLICY ITERATION
Converged at 10-th iteration.
Number of successes: 707/1000
Average number of steps: 75.14568599717114
Executable time: 1.2723543643951416


## Taxi-v3

In [21]:
env = gym.make('Taxi-v3')
print(env.P[0][3])
print(env.observation_space.n, env.action_space.n)

[(1.0, 0, -1, False)]
500 6


In [22]:
# Value Iteration
print('TAXI-V3 USING VALUE ITERATION')
s = time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)
play_multiple_times(env, optimal_policy, 1000)
e = time.time()
print('Executable time:',e-s)

TAXI-V3 USING VALUE ITERATION
Converged at 116-th iteration.
Number of successes: 1000/1000
Average number of steps: 13.057
Executable time: 6.189769983291626


In [23]:
# Policy Iteration
print('TAXI-V3 USING POLICY ITERATION')
s = time.time()
optimal_p_values = policy_iteration(env)[1]
play_multiple_times(env, optimal_p_values, 1000)
e = time.time()
print('Executable time:',e-s)

TAXI-V3 USING POLICY ITERATION
Converged at 28-th iteration.
Number of successes: 1000/1000
Average number of steps: 13.014
Executable time: 3.5976996421813965


# NHẬN XÉT

*   Policy Iteration hội tụ nhanh hơn so với Value Iteration (Policy Iteration sẽ đi tính nhiều giá trị hơn và cho ra nhiều policy hơn nên sẽ nhanh chóng hội tụ so với Value Iteration chỉ đi 1 lần đến khi nào hội tụ)


*   Policy Iteration cũng có thời gian thực thi ngắn hơn so với Value Iteration trong cả ba môi trường


*   Về hiệu quả, cả hai thuật toán đều đạt được số lần thành công tương tự nhau, với một chút lợi thế về số bước trung bình của Policy Iteration trong một số trường hợp.


*   Như vậy, đối với các toy games trong OpenAI Gym, Policy Iteration có xu hướng cho ra kết quả nhanh hơn so với Value Iteration cả về số lần lặp cần thiết để hội tụ và thời gian thực thi trung bình.




